In [1]:
# 1. For i in front/*; do imagemagick/convert 300x300px ...
# 2. Del all images with 2kb (sort, manual, Windows Explorer)
# 3. Basic Preparation Data with bash
#    cat sentencing.csv | sed -e 's/LIFE/100/g' |awk -F\; '{print $1 " " $7}'| awk 'BEGIN {id="id"; yrs="years"} id != $1 {print id","yrs; id=$1; yrs=$2 } id != "id" {yrs = yrs + $2}' >id_years.csv

# Images and Pre-Trained model files are large
rootdir='C:\\Users\\Sandow\\Documents\\CAS AML Project\\cas-aml-prisioners'

import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

os.environ["PATH"] += os.pathsep + 'C:\\Program Files (x86)\\Graphviz\\bin'
os.environ["PATH"] += os.pathsep + 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v10.1\\lib\\x64'
os.environ["CUDA_PATH"] = 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v10.1\\'

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices()

sentence_f='id_years2.csv'
sentence = pd.read_csv(sentence_f)
sentence.head()


Num GPUs Available:  1


,A00147,174
0,A00360,203
1,A00367,83
2,A01054,225
3,A01072,6265
4,A01077,16


In [2]:
sentence.describe()

,174
count,50004.000000
mean,48.072334
std,76.353458
min,0.000000
25%,16.000000
50%,29.000000
75%,55.000000
max,6265.000000


In [3]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[300, 300,3]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(4, activation="softmax")
])
keras.utils.plot_model(model, "my_perpetrators.png", show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [4]:
image_size = (300, 300)
batch_size = 128
directory=os.path.join(rootdir,'Prisioners')

dataset = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
        )
train_ds=dataset.flow_from_directory(
    directory, target_size=image_size, color_mode='rgb', classes=None,
    class_mode='categorical', batch_size=batch_size, seed=42,
    subset="training")
val_ds=dataset.flow_from_directory(
    directory, target_size=image_size, color_mode='rgb', classes=None,
    class_mode='categorical', batch_size=batch_size, seed=42,
    subset="validation")


Found 40021 images belonging to 4 classes.
Found 10004 images belonging to 4 classes.


In [5]:
epochs = 10

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, validation_data=val_ds,
)


Epoch 1/10
313/313 [==============================] - 155s 495ms/step - loss: 5.2713 - accuracy: 0.3102 - val_loss: 5.2846 - val_accuracy: 0.3109
Epoch 2/10
313/313 [==============================] - 161s 514ms/step - loss: 5.2845 - accuracy: 0.3109 - val_loss: 5.2846 - val_accuracy: 0.3109
Epoch 3/10
313/313 [==============================] - 162s 518ms/step - loss: 5.2845 - accuracy: 0.3109 - val_loss: 5.2846 - val_accuracy: 0.3109
Epoch 4/10
313/313 [==============================] - 159s 509ms/step - loss: 5.2845 - accuracy: 0.3109 - val_loss: 5.2846 - val_accuracy: 0.3109
Epoch 5/10
313/313 [==============================] - 163s 522ms/step - loss: 5.2845 - accuracy: 0.3109 - val_loss: 5.2846 - val_accuracy: 0.3109
Epoch 6/10
313/313 [==============================] - 161s 516ms/step - loss: 5.2845 - accuracy: 0.3109 - val_loss: 5.2846 - val_accuracy: 0.3109
Epoch 7/10
313/313 [==============================] - 164s 525ms/step - loss: 5.2845 - accuracy: 0.3109 - val_loss: 5.2846 -